In [11]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

import re
import pandas as pd
from tqdm import tqdm
from loguru import logger
from pymongo import MongoClient
import requests 
import time 
import concurrent.futures

# GitHub Personal Access Token (optional, but recommended for higher rate limits)
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
if not GITHUB_TOKEN:
    logger.warning("GitHub token not found. Rate limits may apply.")

BATCH_SIZE = 500 # Number of records to process before writing to CSV

# Logger config
logger.add("logs/pr-to-issues-keyword.log")


3

# Initial data set


mongoDB connection


In [12]:
# Mongo connection
client = MongoClient("mongodb://localhost:27017/")
db = client.github_data
pull_request_collection = db.pull_requests
logger.info("🔌 MongoDB connected.")

2025-06-03 15:08:35.844 | INFO     | __main__:<module>:5 - 🔌 MongoDB connected.


In [13]:
# Query to filter closed pull requests
qurey = {
    "pull_request.state": "closed",
}

# Fetch all records into a list to prevent cursor timeout during processing
logger.info("Fetching all records from MongoDB...")
all_records_list = list(pull_request_collection.find(qurey))
toal_pull_request_count = len(all_records_list) # Actual number of records fetched
logger.info(f"Successfully fetched {toal_pull_request_count} records.")


2025-06-03 15:08:41.198 | INFO     | __main__:<module>:7 - Fetching all records from MongoDB...
2025-06-03 15:09:04.007 | INFO     | __main__:<module>:10 - Successfully fetched 25042 records.


In [14]:
pr_details_list = [] # List to store dictionaries of PR details

def extract_pr_details(record):
    """
    Extracts relevant details (number, link, title, body) from a pull request record.
    """
    pr_data = record.get("pull_request", {})
    # Handle cases where 'pull_request' key might be missing or empty
    if not pr_data: 
        logger.warning(f"Skipping record due to missing 'pull_request' data. Record ID: {record.get('_id', 'N/A')}")
        return None

    pr_number = pr_data.get("number")
    pr_link = pr_data.get("html_url", None)
    pr_title = pr_data.get("title", None)
    # Use "body" for description as it's the common field for PR description
    pr_body = pr_data.get("body", None) 

    if pr_number is None:
        logger.warning(f"Skipping record due to missing PR number. Record ID: {record.get('_id', 'N/A')}")
        return None

    return {
        "pr_number": pr_number,
        "pr_link": pr_link,
        "pr_title": pr_title,
        "pr_body": pr_body, # "description" in user request, mapped to "body"
    }

# Use ThreadPoolExecutor for processing records concurrently
# os.cpu_count() provides a sensible default for max_workers
# Adjust max_workers based on specific I/O vs CPU characteristics if needed
logger.info(f"Starting extraction of PR details for {toal_pull_request_count} records using concurrent.futures...")

with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    # Submit all tasks to the executor
    future_to_record = {executor.submit(extract_pr_details, record): record for record in all_records_list}
    
    # Process futures as they complete, with a progress bar
    for future in tqdm(concurrent.futures.as_completed(future_to_record), total=len(all_records_list), desc="Extracting PR Details"):
        record_for_future = future_to_record[future] # Get original record for logging in case of error
        try:
            result = future.result()
            if result:  # Add to list if extraction was successful (result is not None)
                pr_details_list.append(result)
        except Exception as exc:
            record_id = record_for_future.get("_id", "N/A")
            logger.error(f"Record ID {record_id} generated an exception during extraction: {exc}")

logger.info(f"Successfully processed all records. Extracted details for {len(pr_details_list)} pull requests.")

2025-06-03 15:09:36.337 | INFO     | __main__:<module>:33 - Starting extraction of PR details for 25042 records using concurrent.futures...
Extracting PR Details: 100%|██████████| 25042/25042 [00:00<00:00, 147144.03it/s]
2025-06-03 15:09:42.987 | INFO     | __main__:<module>:50 - Successfully processed all records. Extracted details for 25042 pull requests.


In [15]:
# Define the path for the output CSV file
output_csv_path = "../data/keyword-linking/pullrequestToIssue/closed_prs_summary.csv"

# Convert the list of dictionaries to a pandas DataFrame and save to CSV
if pr_details_list:
    df_prs = pd.DataFrame(pr_details_list)
    
    # Ensure the columns are in the desired order for the CSV
    # "description" from user request is mapped to "pr_body"
    csv_headers = ["pr_number", "pr_link", "pr_title", "pr_body"]
    df_prs = df_prs[csv_headers]

    try:
        # Save the DataFrame to CSV, overwriting if the file exists
        df_prs.to_csv(output_csv_path, index=False, mode='w')
        logger.info(f"Successfully saved PR details to {output_csv_path}")
    except Exception as e:
        logger.error(f"Failed to save PR details to CSV at {output_csv_path}: {e}")
else:
    logger.warning("No PR details were extracted (or all records were skipped). CSV file will not be created.")

2025-06-03 15:10:25.477 | INFO     | __main__:<module>:16 - Successfully saved PR details to ../data/keyword-linking/pullrequestToIssue/closed_prs_summary.csv


# Key word processing


## Title


In [14]:
import pandas as pd
import re
from loguru import logger
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from typing import List, Tuple, Optional, Dict
import time


# Thread-safe lock for writing results
write_lock = threading.Lock()

csv_path = "../data/keyword-linking/pullrequestToIssue/closed_prs_summary.csv"

def extract_all_issue_numbers_from_title(title: str) -> List[Dict[str, str]]:
    """
    Extracts all issue numbers from pull request title.
    Returns a list of dictionaries with issue_number and issue_link (if found as URL).
    """
    if pd.isna(title) or not title.strip():
        return []

    # Covers: fix(es/ed/ing), clos(e/es/ed), resolv(e/es/ed), issue(s), Backport, Revert(s)
    keyword_pattern = r'\b(?:fix(?:es|ed|ing)?|clos(?:e|es|ed)|resolv(?:e|es|ed)|issue|Backport|Revert(?:s)?)\b'

    # Comprehensive patterns to find issue numbers, ordered by specificity
    issue_patterns = [
        # Priority 1: Keywords followed by colon and GitHub URL (most specific)
        rf'{keyword_pattern}:\s+(https?://github\.com/[^/\s]+/[^/\s]+/(?:issues|pull)/(\d+))',
        
        # Priority 2: Keywords followed by full GitHub issue URL
        rf'{keyword_pattern}\s+(https?://github\.com/[^/\s]+/[^/\s]+/(?:issues|pull)/(\d+))',
        
        # Priority 3: Keywords followed by repo path and issue number
        rf'{keyword_pattern}\s+[^/\s]+/[^/\s]+/issues/(\d+)',
        
        # Priority 4: Keywords + <text> + GitHub issue links + optional plain number
        rf'{keyword_pattern}.*<.*?>\s+(https?://github\.com/[^/\s]+/[^/\s]+/(?:issues|pull)/(\d+))(?:\s+(\d+))*',
        
        # Priority 5: Keywords + <text> + multiple issue numbers
        rf'{keyword_pattern}.*<.*?>\s+#(\d+)(?:\s+#(\d+))*',
        
        # Priority 6: Keywords followed by square brackets
        rf'{keyword_pattern}.*\[#(\d+)\]',
        
        # Priority 7: Square brackets followed by keywords
        rf'\[#(\d+)\].*{keyword_pattern}',
        
        # Priority 8: Keywords followed by (from #number)
        rf'{keyword_pattern}.*\(from\s+#(\d+)\)',
        
        # Priority 9: Keywords followed by "part of #number" or "part of issue #number"
        rf'{keyword_pattern}.*part\s+of\s+(?:issue\s+)?#(\d+)',
        
        # Priority 10: Keywords followed by "/fix-#number" or "/fix-number"
        rf'{keyword_pattern}.*(?:/fix-#(\d+)|/fix-(\d+))',
        
        # Priority 11: Multiple issues in one line (e.g., "fixes #123, #456, #789")
        rf'{keyword_pattern}[^#]*?((?:#\d+(?:\s*,\s*#\d+)*)+)',
        
        # Priority 12: Keywords followed by any text and #number
        rf'{keyword_pattern}.*#(\d+)',
        
        # Priority 13: Keywords followed by #number (direct)
        rf'{keyword_pattern}\s+#(\d+)',
        
        # Priority 14: Keywords followed by plain number (no #)
        rf'{keyword_pattern}\s+(\d+)\b',
        
        # Priority 15: #number followed by keyword
        rf'#(\d+)\s+{keyword_pattern}',
        
        # Priority 16: Specific "Address review suggestions" pattern
        rf'Address\s+review\s+suggestions.*#(\d+)',
        
        # Priority 17: Multiple issue numbers after "issues" (e.g., "issues #1159 #947")
        r'\bissues?\s+#(\d+)(?:\s+#(\d+))*',
        
        # Priority 18: GitHub URLs without keywords (issues only, not pull requests)
        r'(https?://github\.com/[^/\s]+/[^/\s]+/issues/(\d+))',
    ]

    found_issues = []
    
    for pattern in issue_patterns:
        matches = re.finditer(pattern, title, re.IGNORECASE)
        for match in matches:
            groups = match.groups()
            
            # Check if this is a URL pattern (contains full GitHub URL)
            full_url = None
            issue_number = None
            
            for group in groups:
                if group and group.startswith('http'):
                    full_url = group
                elif group and group.isdigit():
                    issue_number = group
                    
            if full_url and issue_number:
                # Found a full URL with issue number
                if not any(item['issue_number'] == issue_number for item in found_issues):
                    found_issues.append({
                        'issue_number': issue_number,
                        'issue_link': full_url
                    })
                continue
            
            # Handle non-URL patterns
            for group in groups:
                if group:
                    # Handle multiple issues in one match (e.g., "#123, #456")
                    if '#' in group:
                        issue_nums = re.findall(r'#(\d+)', group)
                        for issue_num in issue_nums:
                            if not any(item['issue_number'] == issue_num for item in found_issues):
                                found_issues.append({
                                    'issue_number': issue_num,
                                    'issue_link': None
                                })
                    elif group.isdigit():
                        if not any(item['issue_number'] == group for item in found_issues):
                            found_issues.append({
                                'issue_number': group,
                                'issue_link': None
                            })
    
    return found_issues

def generate_issue_link(issue_number: str, pr_link: str) -> str:
    """
    Generate GitHub issue link based on issue number and PR link.
    Assumes same repository as the PR.
    """
    if not issue_number or not pr_link:
        return ""
    
    # Extract repository info from PR link
    # Example: https://github.com/owner/repo/pull/123 -> https://github.com/owner/repo/issues/456
    match = re.match(r'(https?://github\.com/[^/]+/[^/]+)/', pr_link)
    if match:
        repo_base = match.group(1)
        return f"{repo_base}/issues/{issue_number}"
    
    return ""

def process_row(row_data: Tuple[int, pd.Series]) -> List[dict]:
    """
    Process a single row and return list of records (one per issue found).
    """
    idx, row = row_data
    
    try:
        title = str(row.get('pr_title', ''))
        pr_link = str(row.get('pr_link', ''))
        
        # Extract all issue numbers from title with their URLs (if found)
        issue_data = extract_all_issue_numbers_from_title(title)
        
        results = []
        
        if issue_data:
            # Create separate record for each issue found
            for issue_info in issue_data:
                issue_number = issue_info['issue_number']
                found_url = issue_info['issue_link']
                
                # Use found URL if available, otherwise generate one
                if found_url:
                    issue_link = found_url
                else:
                    issue_link = generate_issue_link(issue_number, pr_link)
                
                # Create new record with all original data plus extracted info
                new_record = row.to_dict()
                new_record.update({
                    'extracted_issue_number': issue_number,
                    'extracted_issue_link': issue_link,
                    'found_in_title': True,
                    'original_row_index': idx
                })
                results.append(new_record)
                
            logger.info(f"Row {idx}: Found {len(issue_data)} issues in title: {[item['issue_number'] for item in issue_data]}")
        else:
            # No issues found, keep original record with null values
            new_record = row.to_dict()
            new_record.update({
                'extracted_issue_number': None,
                'extracted_issue_link': None,
                'found_in_title': False,
                'original_row_index': idx
            })
            results.append(new_record)
            
        return results
        
    except Exception as e:
        logger.error(f"Error processing row {idx}: {str(e)}")
        # Return original record with error info
        new_record = row.to_dict()
        new_record.update({
            'extracted_issue_number': None,
            'extracted_issue_link': None,
            'found_in_title': False,
            'original_row_index': idx,
            'processing_error': str(e)
        })
        return [new_record]

def run():
    """
    Main function to process the CSV and create new output file.
    """
    logger.info("Starting issue number extraction process")
    
    try:
        # Load the CSV file
        logger.info(f"Loading CSV file: {csv_path}")
        df = pd.read_csv(csv_path)
        logger.info(f"Loaded {len(df)} records from CSV")
        
        # Prepare data for parallel processing
        row_data = list(df.iterrows())
        
        # Process rows in parallel
        all_results = []
        
        logger.info("Starting parallel processing...")
        with ThreadPoolExecutor(max_workers=4) as executor:
            # Submit all tasks
            future_to_row = {executor.submit(process_row, row): row[0] for row in row_data}
            
            # Process completed tasks with progress bar
            for future in tqdm(as_completed(future_to_row), total=len(future_to_row), desc="Processing rows"):
                try:
                    row_results = future.result()
                    with write_lock:
                        all_results.extend(row_results)
                except Exception as e:
                    row_idx = future_to_row[future]
                    logger.error(f"Failed to process row {row_idx}: {str(e)}")
        
        # Create new DataFrame from results
        logger.info(f"Creating new DataFrame with {len(all_results)} records")
        new_df = pd.DataFrame(all_results)
        
        # Generate output filename
        output_path = csv_path.replace('.csv', '_with_extracted_issues.csv')
        
        # Save to new CSV
        logger.info(f"Saving results to: {output_path}")
        new_df.to_csv(output_path, index=False)
        
        # Log statistics
        total_original_rows = len(df)
        total_new_rows = len(new_df)
        rows_with_issues = len(new_df[new_df['found_in_title'] == True])
        unique_issues_found = len(new_df[new_df['extracted_issue_number'].notna()]['extracted_issue_number'].unique())
        
        # Additional statistics for link sources
        extracted_links = len(new_df[(new_df['extracted_issue_link'].notna()) & 
                                    (new_df['extracted_issue_link'].str.startswith('http'))])
        generated_links = total_new_rows - extracted_links - len(new_df[new_df['extracted_issue_link'].isna()])
        
        logger.info("="*50)
        logger.info("PROCESSING SUMMARY")
        logger.info("="*50)
        logger.info(f"Original rows: {total_original_rows}")
        logger.info(f"New rows: {total_new_rows}")
        logger.info(f"Rows with issues found: {rows_with_issues}")
        logger.info(f"Unique issues extracted: {unique_issues_found}")
        logger.info(f"Links used from title: {extracted_links}")
        logger.info(f"Links generated: {generated_links}")
        logger.info(f"Expansion ratio: {total_new_rows/total_original_rows:.2f}x")
        logger.info(f"Output file: {output_path}")
        logger.info("="*50)
        
        print(f"\n✅ Processing completed successfully!")
        print(f"📊 Original rows: {total_original_rows}")
        print(f"📊 New rows: {total_new_rows}")
        print(f"📊 Rows with issues: {rows_with_issues}")
        print(f"📊 Unique issues found: {unique_issues_found}")
        print(f"🔗 Links from title: {extracted_links}")
        print(f"🔗 Generated links: {generated_links}")
        print(f"💾 Output saved to: {output_path}")
        
        return new_df
        
    except Exception as e:
        logger.error(f"Fatal error in main process: {str(e)}")
        raise

run()

2025-06-06 15:18:02.008 | INFO     | __main__:run:216 - Starting issue number extraction process
2025-06-06 15:18:02.012 | INFO     | __main__:run:220 - Loading CSV file: ../data/keyword-linking/pullrequestToIssue/closed_prs_summary.csv
2025-06-06 15:18:02.370 | INFO     | __main__:run:222 - Loaded 25042 records from CSV
2025-06-06 15:18:03.987 | INFO     | __main__:run:230 - Starting parallel processing...
2025-06-06 15:18:04.015 | INFO     | __main__:process_row:185 - Row 41: Found 1 issues in title: ['12492']
2025-06-06 15:18:04.026 | INFO     | __main__:process_row:185 - Row 89: Found 1 issues in title: ['12492']
2025-06-06 15:18:04.033 | INFO     | __main__:process_row:185 - Row 131: Found 1 issues in title: ['12584']
2025-06-06 15:18:04.045 | INFO     | __main__:process_row:185 - Row 218: Found 1 issues in title: ['3440']
2025-06-06 15:18:04.054 | INFO     | __main__:process_row:185 - Row 239: Found 1 issues in title: ['4581']
2025-06-06 15:18:04.056 | INFO     | __main__:process


✅ Processing completed successfully!
📊 Original rows: 25042
📊 New rows: 25065
📊 Rows with issues: 313
📊 Unique issues found: 291
🔗 Links from title: 313
🔗 Generated links: 0
💾 Output saved to: ../data/keyword-linking/pullrequestToIssue/closed_prs_summary_with_extracted_issues.csv


,pr_number,pr_link,pr_title,pr_body,extracted_issue_number,extracted_issue_link,found_in_title,original_row_index
0,3160,https://github.com/ballerina-platform/ballerin...,Update web socket sample for action invoation ...,NaN,None,None,False,10799
1,42658,https://github.com/ballerina-platform/ballerin...,Create resource jars separately to read the re...,"## Purpose\r\n> Describe the problems, issues,...",None,None,False,7855
2,8108,https://github.com/ballerina-platform/ballerin...,Fix issues in list command help,## Purpose\r\n> This PR fixes issues in list c...,None,None,False,7854
3,43690,https://github.com/ballerina-platform/ballerin...,Update full build pipeline of PR build,## Purpose\r\n> $title.\r\n\r\nFixes #<Issue N...,None,None,False,7853
4,9535,https://github.com/ballerina-platform/ballerin...,Update keyword for composer syntax highlighting,## Purpose\r\nUpdate entries keywords in monar...,None,None,False,7852
...,...,...,...,...,...,...,...,...
25060,8235,https://github.com/ballerina-platform/ballerin...,Fix WebSub doc issues,## Purpose\r\nFix WebSub doc issues,None,None,False,17195
25061,12351,https://github.com/ballerina-platform/ballerin...,Update BBE with sync send and flush,## Purpose\r\n> Add sync send and flush to the...,None,None,False,14039
25062,25356,https://github.com/ballerina-platform/ballerin...,"Fix New Line Formatting in ""java.jdbc"" API Docs",## Purpose\r\nFix the new line formatting in j...,None,None,False,22298
25063,39163,https://github.com/ballerina-platform/ballerin...,Move architecture model generator implementati...,## Purpose\r\n> This PR removes the architectu...,None,None,False,14038


## Discription

In [13]:
import pandas as pd
import os  
import re
from loguru import logger
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from typing import List, Tuple, Optional
import time

def extract_all_issue_numbers_from_text(text: str) -> List[str]:
    """
    Extracts all issue numbers from a given text string.
    
    This function uses two methods:
    1. A list of regular expressions to find standard issue references (e.g., "fixes #123").
    2. A specific procedural search for lists of issues that appear after the phrase 
       "Following issues".
       
    Returns a list of unique issue numbers found.
    """
    if pd.isna(text) or not text.strip():
        return []

    found_issues = set()  # Use a set to automatically handle duplicates

    # --- Part 1: General Regex-Based Extraction (Your Original Method) ---
    
    # Covers: fix(es/ed/ing), clos(e/es/ed), resolv(e/es/ed), issue(s)
    keyword_pattern = r'\b(?:fix(?:es|ed|ing)?|clos(?:e|es|ed)|resolv(?:e|es|ed)|issue(?:s)?)\b'

    issue_patterns = [
        # Matches: keyword + full GitHub issue/PR URL
        rf'{keyword_pattern}\s+https?://github\.com/[^/]+/[^/]+/(?:issues|pull)/(\d+)',
        # Matches: keyword + #issue_number (e.g., "closes #456", "Fixes #32160")
        rf'{keyword_pattern}\s+#(\d+)\b',
        # Matches: keyword + issue_number
        rf'{keyword_pattern}\s+(\d+)\b',
        # Matches: keyword + text + (from #issue_number)
        rf'{keyword_pattern}.*\(from\s+#(\d+)\)',
        # Matches: keyword + text + #issue_number (a bit broad, but can be useful)
        rf'{keyword_pattern}.*#(\d+)\b',
        # Matches: keyword + text + (part of #issue_number)
        rf'{keyword_pattern}.*(?:part of\s+(?:issue\s+)?#(\d+)\b)',
        # Matches: keyword + text + /fix-#issue_number
        rf'{keyword_pattern}.*(?:/fix-#(\d+)\b|/fix-(\d+)\b)',
        # Matches: "Fixes: " + full GitHub issue URL
        rf'Fixes:\s+https?://github\.com/[^/]+/[^/]+/issues/(\d+)',
        # Matches: keyword + multiple #issue_numbers separated by commas
        rf'{keyword_pattern}[^#]*?((?:#\d+(?:\s*,\s*#\d+)*)+)',
        # Matches: standalone full GitHub issue URL
        r'https?://github\.com/[^/]+/[^/]+/issues/(\d+)',
        # Matches: "Fix " + org/repo/issues/issue_number
        rf'Fix\s+[^/]+/[^/]+/issues/(\d+)',
        # General fallback for any standalone #<number> not caught by other patterns
        r'#(\d+)\b'
    ]

    for pattern in issue_patterns:
        # Using re.IGNORECASE to match keywords like "Fixes" or "fixes"
        matches = re.finditer(pattern, text, re.IGNORECASE)
        for match in matches:
            for group_val in match.groups():
                if group_val:
                    # Handle multiple issues in one match (e.g., "#123, #456")
                    if '#' in group_val:
                        issue_nums_in_group = re.findall(r'#(\d+)', group_val)
                        found_issues.update(issue_nums_in_group)
                    else:
                        found_issues.add(group_val)

    # --- Part 2: Procedural Extraction for "Following issues" List ---
    
    is_in_remarks_section = False
    for line in text.splitlines():
        # Turn on the flag when the trigger phrase is found
        if "Following issues" in line:
            is_in_remarks_section = True
            continue

        # If the flag is on, start extracting numbers
        if is_in_remarks_section:
            # If the line is empty or starts a new section, the list is over
            if not line.strip() or line.strip().startswith("##"):
                break

            # Find all #<number> patterns on the line and add to the set
            found_in_line = re.findall(r'#(\d+)\b', line)
            if found_in_line:
                found_issues.update(found_in_line)
    
    return sorted(list(found_issues), key=int) # Return a sorted list

def process_row_for_body(row_data: Tuple[int, pd.Series]) -> List[dict]:
    """
    Process a single row to extract issue numbers from 'pr_body'.
    Returns a list of dictionaries, one for each issue found, or one original if none found.
    """
    idx, row = row_data
    
    try:
        pr_body = str(row.get('pr_body', ''))
        pr_link = str(row.get('pr_link', ''))
        
        # Extract all issue numbers from pr_body
        issue_numbers = extract_all_issue_numbers_from_text(pr_body)
        
        results = []
        
        if issue_numbers:
            # Create separate record for each issue found in body
            for issue_num in issue_numbers:
                issue_link = generate_issue_link(issue_num, pr_link) # generate_issue_link from CELL 11
                
                new_record = row.to_dict()
                new_record.update({
                    'extracted_issue_number_body': issue_num,
                    'extracted_issue_link_body': issue_link,
                    'found_in_body': True,
                    'original_row_index': idx  # Keep track of the original row
                })
                results.append(new_record)
                
            logger.info(f"Row {idx}: Found {len(issue_numbers)} issues in body: {issue_numbers}")
        else:
            # No issues found in body, keep original record with null/false values for body extraction
            new_record = row.to_dict()
            new_record.update({
                'extracted_issue_number_body': None,
                'extracted_issue_link_body': None,
                'found_in_body': False,
                'original_row_index': idx
            })
            results.append(new_record)
            
        return results
        
    except Exception as e:
        logger.error(f"Error processing row {idx} for body extraction: {str(e)}")
        # Return original record with error info for body extraction
        new_record = row.to_dict()
        new_record.update({
            'extracted_issue_number_body': None,
            'extracted_issue_link_body': None,
            'found_in_body': False,
            'original_row_index': idx,
            'processing_error_body': str(e)
        })
        return [new_record]

def run_body_extraction():
    """
    Main function to process the CSV for issues in 'pr_body' and create a new output file.
    """
    logger.info("Starting issue number extraction from PR BODY content")
    
    try:
        # csv_path is expected to be defined from a previous cell (e.g., CELL 11)
        # Defaulting if not found, but ideally it's already set.
        current_csv_path = csv_path if 'csv_path' in globals() else "../data/keyword-linking/pullrequestToIssue/closed_prs_summary.csv"

        logger.info(f"Loading CSV file: {current_csv_path}")
        df = pd.read_csv(current_csv_path)
        logger.info(f"Loaded {len(df)} records from CSV for body extraction")
        
        # Prepare data for parallel processing
        row_data_for_body = list(df.iterrows())
        
        all_results_body = []
        
        logger.info("Starting parallel processing for PR bodies...")
        # Using max_workers=4 for consistency with CELL 11, adjust if needed
        with ThreadPoolExecutor(max_workers= os.cpu_count()) as executor:
            future_to_row_body = {executor.submit(process_row_for_body, row_bd): row_bd[0] for row_bd in row_data_for_body}
            
            for future_bd in tqdm(as_completed(future_to_row_body), total=len(future_to_row_body), desc="Processing PR Bodies"):
                try:
                    row_results_bd = future_bd.result()
                    # write_lock is expected to be defined from a previous cell (e.g., CELL 11)
                    with write_lock: 
                        all_results_body.extend(row_results_bd)
                except Exception as e_bd:
                    row_idx_bd = future_to_row_body[future_bd]
                    logger.error(f"Failed to process row {row_idx_bd} for body extraction in main loop: {str(e_bd)}")
        
        logger.info(f"Creating new DataFrame with {len(all_results_body)} records from body extraction")
        new_df_body = pd.DataFrame(all_results_body)
        
        # Generate output filename for body extraction results
        output_path_body = current_csv_path.replace('.csv', '_with_extracted_issues_from_body.csv')
        
        logger.info(f"Saving body extraction results to: {output_path_body}")
        new_df_body.to_csv(output_path_body, index=False)
        
        # Log statistics for body extraction
        total_original_rows_bd = len(df)
        total_new_rows_bd = len(new_df_body)
        rows_with_issues_in_body = len(new_df_body[new_df_body['found_in_body'] == True])
        unique_issues_found_in_body = 0
        if 'extracted_issue_number_body' in new_df_body.columns:
             unique_issues_found_in_body = len(new_df_body[new_df_body['extracted_issue_number_body'].notna()]['extracted_issue_number_body'].unique())
        
        logger.info("="*50)
        logger.info("PR BODY EXTRACTION PROCESSING SUMMARY")
        logger.info("="*50)
        logger.info(f"Original rows processed: {total_original_rows_bd}")
        logger.info(f"New rows generated (from body): {total_new_rows_bd}")
        logger.info(f"Rows with issues found in body: {rows_with_issues_in_body}")
        logger.info(f"Unique issues extracted from body: {unique_issues_found_in_body}")
        if total_original_rows_bd > 0:
            logger.info(f"Expansion ratio (body): {total_new_rows_bd/total_original_rows_bd:.2f}x")
        logger.info(f"Output file (body extraction): {output_path_body}")
        logger.info("="*50)
        
        print(f"\n✅ PR Body issue extraction completed successfully!")
        print(f"📊 Original rows: {total_original_rows_bd}")
        print(f"📊 New rows (from body): {total_new_rows_bd}")
        print(f"📊 Rows with issues found in body: {rows_with_issues_in_body}")
        print(f"📊 Unique issues found in body: {unique_issues_found_in_body}")
        print(f"💾 Output saved to: {output_path_body}")
        
        return new_df_body
        
    except FileNotFoundError:
        logger.error(f"Error: The CSV file was not found at {current_csv_path}.")
        print(f"❌ Error: The CSV file was not found at {current_csv_path}.")
    except Exception as e:
        logger.error(f"Fatal error in PR body extraction process: {str(e)}")
        print(f"❌ Fatal error in PR body extraction process: {str(e)}")
        raise

# Execute the body extraction process
# This will use csv_path, generate_issue_link, and write_lock from the previous cells' scope.
result_df_body = run_body_extraction()

2025-06-06 11:44:04.202 | INFO     | __main__:run_body_extraction:155 - Starting issue number extraction from PR BODY content
2025-06-06 11:44:04.210 | INFO     | __main__:run_body_extraction:162 - Loading CSV file: ../data/keyword-linking/pullrequestToIssue/closed_prs_summary.csv
2025-06-06 11:44:04.672 | INFO     | __main__:run_body_extraction:164 - Loaded 25042 records from CSV for body extraction
2025-06-06 11:44:06.760 | INFO     | __main__:run_body_extraction:171 - Starting parallel processing for PR bodies...
2025-06-06 11:44:06.772 | INFO     | __main__:process_row_for_body:124 - Row 16: Found 4 issues in body: ['6490', '6882', '6883', '6892']
2025-06-06 11:44:06.780 | INFO     | __main__:process_row_for_body:124 - Row 29: Found 1 issues in body: ['16625']
2025-06-06 11:44:06.780 | INFO     | __main__:process_row_for_body:124 - Row 31: Found 1 issues in body: ['29017']
2025-06-06 11:44:06.780 | INFO     | __main__:process_row_for_body:124 - Row 32: Found 1 issues in body: ['257


✅ PR Body issue extraction completed successfully!
📊 Original rows: 25042
📊 New rows (from body): 30058
📊 Rows with issues found in body: 15565
📊 Unique issues found in body: 11253
💾 Output saved to: ../data/keyword-linking/pullrequestToIssue/closed_prs_summary_with_extracted_issues_from_body.csv
